# Predições Privadas com Keras Syft

## Passo 3: Predições Privadas usando Syft Keras - Instanciando um cliente

Parabéns! Após treinar o seu modelo com Keras usual e assegurá-lo com Syft Keras, você está pronto para solicitar algumas previsões privadas.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import syft as sy

## Dados

Aqui, pré-processaremos nossos dados do MNIST. De forma idêntica a como pré-processamos durante o treinamento.

In [2]:
# dimensoes da imagem de entrada
img_rows, img_cols = 28, 28

# os dados, divididos em conjuntos de treino e teste
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Conectar ao modelo

Antes de consultar o modelo, você deve conectar-se a ele. Para isso, você deve criar um cliente. Em seguida, defina exatamente os três TFEWorkers (`alice`, `bob`, e `carol`) e agrupe-os. Finalmente, chame `connect_to_model`. Isso cria um servidor de enfileiramento TFE no lado do cliente que se conecta ao servidor de enfileiramento configurado por `model.serve()`na **Parte 13b**. A fila será responsável pelo compartilhamento secreto dos dados em texto sem formatação antes de enviar os compartilhamentos em uma solicitação de predição.

In [3]:
num_classes = 10
input_shape = (1, 28, 28, 1)
output_shape = (1, num_classes)

In [4]:
client = sy.TFEWorker()

alice = sy.TFEWorker(host='localhost:4000')
bob = sy.TFEWorker(host='localhost:4001')
carol = sy.TFEWorker(host='localhost:4002')
cluster = sy.TFECluster(alice, bob, carol)

client.connect_to_model(input_shape, output_shape, cluster)

INFO:tf_encrypted:Starting session on target 'grpc://localhost:4000' using config graph_options {
}



## Modelo de Consulta

Você está pronto para receber algumas predições privadas! Chamando `query_model`, você irá inserir uma `image`na fila criada acima, compartilhar secretamente os dados localmente, e enviar os compartilhamentos para o modelo servidor na **Parte 13b**.

In [5]:
# entradas do usuario
num_tests = 3
images, expected_labels = x_test[:num_tests], y_test[:num_tests]

In [6]:
for image, expected_label in zip(images, expected_labels):

    res = client.query_model(image.reshape(1, 28, 28, 1))
    predicted_label = np.argmax(res)

    print("The image had label {} and was {} classified as {}".format(
        expected_label,
        "correctly" if expected_label == predicted_label else "wrongly",
        predicted_label))

The image had label 7 and was correctly classified as 7
The image had label 2 and was correctly classified as 2
The image had label 1 and was correctly classified as 1


Isso é ótimo. Você é capaz de classificar essas três imagens corretamente! Mas o que há de especial nessas predições é que você não revela nenhuma informação privada para obter este serviço. O modelo instanciado nunca viu seus dados de entrada ou suas predições, e você nunca fez o download do modelo. Você foi capaz de obter previsões privadas sobre dados com um modelo criptografado!

Antes de nos adiantarmos para aplicar isso em nossos próprios aplicativos, vamos voltar rapidamente à **Parte 13b** para limpar nosso modelo servido!
